# TensorFlow ArcFace

### Ref https://github.com/peteryuX/arcface-tf2

In [1]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from models import ArcFaceModel
# from losses import SoftmaxLoss
from losses import softmax_loss
import dataset
import tensorflow as tf
import os
import logging

tf.get_logger().setLevel(logging.ERROR)
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
!echo $CUDA_VISIBLE_DEVICES

2


In [3]:
### IJB-C Dataset

# batch_size = 128
# input_size = 112
# embd_shape = 512
# head_type = 'ArcHead'
# backbone_type = 'MobileNetV2'
# w_decay=5e-4
# num_classes = 3584 
# base_lr = 0.01
# dataset_len = 13033 
# epochs = 100
# steps_per_epoch = dataset_len // batch_size

### MS1M dataset

batch_size = 128 # Initially 128
input_size = 112
embd_shape = 512
head_type = 'ArcHead'
backbone_type = 'EfficientNetLite0' # MobileNetV2, ResNet50, EfficientNetLite0~6
w_decay=5e-4
num_classes = 85742 
dataset_len = 5822653 
base_lr = 0.01 # initially 0.01
epochs = 20
save_steps = 1000
train_size = int(0.8 * dataset_len)
print(train_size)
steps_per_epoch = train_size // batch_size
print(steps_per_epoch)
val_size = dataset_len - train_size
print(val_size)
validation_steps = val_size // batch_size
print(validation_steps)
steps = 1
is_ccrop=False
binary_img=True
is_Adam = False

dgx = True

4658122
36391
1164531
9097


In [5]:
# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'
if dgx:
    train_data_dir = "/raid/workspace/jbpark/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'
else:
    train_data_dir = "/hd/jbpark/dataset/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'


train_dataset, val_dataset = dataset.load_tfrecord_dataset(
    tfrecord_name, batch_size, train_size=train_size, binary_img=binary_img, 
    is_ccrop=is_ccrop)

if dgx:
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:2"])
else:
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0"])
#     strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

with strategy.scope():
    model = ArcFaceModel(size=input_size,
                             backbone_type=backbone_type,
                             num_classes=num_classes,
                             head_type=head_type,
                             embd_shape=embd_shape,
                             w_decay=w_decay,
                             training=True)
    model.summary()

    learning_rate = tf.constant(base_lr)
    if is_Adam:
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=learning_rate, momentum=0.9, nesterov=True)
    else:
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=learning_rate)

    loss_fn = softmax_loss

    model.compile(optimizer=optimizer, loss=loss_fn)

Model: "arcface_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
efficientnet-lite0 (Functional) (None, 4, 4, 1280)   3413031     input_image[0][0]                
__________________________________________________________________________________________________
OutputLayer (Functional)        (None, 512)          10493440    efficientnet-lite0[0][0]         
__________________________________________________________________________________________________
label (InputLayer)              [(None,)]            0                                            
______________________________________________________________________________________

In [6]:
from pathlib import Path

if dgx:
    base_dir = "/raid/workspace/jbpark/FaceRecognition/"
    # save_name = "ms1m_mobilenet_check/"
    save_name = "ms1m_efficientnet_check/"
else:
    base_dir = "/hd/jbpark/models/"
    save_name = "ms1m_check/"

if is_Adam:
    version = "Adam"
else:
    version = "SGD"

Path(base_dir + 'checkpoints/w_tfidentity/' + save_name + version).mkdir(parents=True, exist_ok=True)

### MS1M dataset
tb_callback = TensorBoard(log_dir='logs/arcface/',
                                  update_freq = batch_size * 5,
                                  profile_batch=0)
tb_callback._total_batches_seen = steps
tb_callback._samples_seen = steps * batch_size
mc_callback = ModelCheckpoint(
            base_dir + 'checkpoints/w_tfidentity/' + save_name + version +'/e_{epoch}_l_{loss}.ckpt',
            save_freq = save_steps, verbose=1,
            save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=0.1, min_lr=0.001)
callbacks = [mc_callback, tb_callback,reduce_lr]

### IJB-C Dataset
# callbacks = [
#     ModelCheckpoint(
#         base_dir+"checkpoints/"+save_name+".ckpt", 
# #         monitor='val_accuracy', 
#         monitor='loss', 
#         verbose=1, 
#         save_best_only=True, 
#         save_weights_only = True,
#         mode='min'
#     ),
#     EarlyStopping(
# #         monitor='val_accuracy', 
#         monitor='loss', 
#         patience=15, 
#         min_delta=0.001, 
#         mode='min'
#     )
# ]

model.fit(
    train_dataset, 
    validation_data=val_dataset,
    epochs=epochs, 
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)

Epoch 1/20
 1000/36391 [..............................] - ETA: 1:25:07 - loss: 46.5150
Epoch 00001: saving model to /raid/workspace/jbpark/FaceRecognition/checkpoints/w_tfidentity/ms1m_efficientnet_check/SGD/e_1_l_46.50857162475586.ckpt
 2000/36391 [>.............................] - ETA: 1:22:37 - loss: 46.4989
Epoch 00001: saving model to /raid/workspace/jbpark/FaceRecognition/checkpoints/w_tfidentity/ms1m_efficientnet_check/SGD/e_1_l_46.447227478027344.ckpt
 3000/36391 [=>............................] - ETA: 1:19:58 - loss: 46.4639
Epoch 00001: saving model to /raid/workspace/jbpark/FaceRecognition/checkpoints/w_tfidentity/ms1m_efficientnet_check/SGD/e_1_l_46.33393859863281.ckpt
 4000/36391 [==>...........................] - ETA: 1:17:26 - loss: 46.4115
Epoch 00001: saving model to /raid/workspace/jbpark/FaceRecognition/checkpoints/w_tfidentity/ms1m_efficientnet_check/SGD/e_1_l_46.16621017456055.ckpt
 5000/36391 [===>..........................] - ETA: 1:14:56 - loss: 46.3424
Epoch 00

KeyboardInterrupt: 

### Resume training with latest checkpoint

In [4]:
from glob import glob
if dgx:
    base_dir = "/raid/workspace/jbpark/FaceRecognition/checkpoints/w_tfidentity/"
    # save_name = "ms1m_mobilenet_check/SGD/*"
    save_name = "ms1m_efficientnet_check/SGD/*"
else:
    base_dir = "/hd/jbpark/models/checkpoints/w_tfidentity/"
    save_name = "ms1m_check/SGD/*"
file_list = []
for files in glob(base_dir+save_name):
    file_list.append(files.split('/')[-1].split('l_')[-1])
file_list.sort()

load_file_name = []
for files in glob(base_dir+save_name):
    if file_list[0] == files.split('/')[-1].split('l_')[-1]:
        load_file_name = files
best_checkpoint = load_file_name.split('.data')[0]
initial_epoch = int(load_file_name.split('e_')[-1].split('_')[0])-1
print(initial_epoch)
print(best_checkpoint)

0
/raid/workspace/jbpark/FaceRecognition/checkpoints/w_tfidentity/ms1m_efficientnet_check/SGD/e_1_l_38.433876037597656.ckpt


In [5]:
from pathlib import Path

if dgx:
    base_dir = "/raid/workspace/jbpark/FaceRecognition/"
    # save_name = "ms1m_mobilenet_check/"
    save_name = "ms1m_efficientnet_check/"
else:
    base_dir = "/hd/jbpark/models/"
    save_name = "ms1m_check/"

if is_Adam:
    version = "Adam"
else:
    version = "SGD"

Path(base_dir + 'checkpoints/w_tfidentity/' + save_name + version).mkdir(parents=True, exist_ok=True)

# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'

if dgx:
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:2"])
else:
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0"])
#     strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

if dgx:
    train_data_dir = "/raid/workspace/jbpark/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'
else:
    train_data_dir = "/hd/jbpark/dataset/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'


train_dataset, val_dataset = dataset.load_tfrecord_dataset(
    tfrecord_name, batch_size, train_size=train_size, binary_img=binary_img,
    is_ccrop=is_ccrop)

with strategy.scope():

    model = ArcFaceModel(size=input_size,
                             backbone_type=backbone_type,
                             num_classes=num_classes,
                             head_type=head_type,
                             embd_shape=embd_shape,
                             w_decay=w_decay,
                             training=True)
    model.load_weights(best_checkpoint)

    learning_rate = tf.constant(base_lr)
    if is_Adam:
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=learning_rate, momentum=0.9, nesterov=True)
    else:
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=learning_rate)

    loss_fn = softmax_loss

    model.compile(optimizer=optimizer, loss=loss_fn)
    model.summary()
    
tb_callback = TensorBoard(log_dir='logs/arcface/',
                                  update_freq = batch_size * 5,
                                  profile_batch=0)
tb_callback._total_batches_seen = steps
tb_callback._samples_seen = steps * batch_size
mc_callback = ModelCheckpoint(
            base_dir + 'checkpoints/w_tfidentity/' + save_name + version +'/e_{epoch}_l_{loss}.ckpt',
            save_freq = save_steps, verbose=1,
            save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.001)
callbacks = [mc_callback, tb_callback,reduce_lr]

Model: "arcface_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
efficientnet-lite0 (Functional) (None, 4, 4, 1280)   3413031     input_image[0][0]                
__________________________________________________________________________________________________
OutputLayer (Functional)        (None, 512)          10493440    efficientnet-lite0[0][0]         
__________________________________________________________________________________________________
label (InputLayer)              [(None,)]            0                                            
______________________________________________________________________________________

In [ ]:
model.fit(
    train_dataset, 
    validation_data=val_dataset,
    epochs=epochs, 
    initial_epoch=initial_epoch,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)

Epoch 1/20
 1000/36391 [..............................] - ETA: 1:34:10 - loss: 37.4833- ETA
Epoch 00001: saving model to /raid/workspace/jbpark/FaceRecognition/checkpoints/w_tfidentity/ms1m_efficientnet_check/SGD/e_1_l_37.47681427001953.ckpt
 2000/36391 [>.............................] - ETA: 1:33:45 - loss: 37.4804
Epoch 00001: saving model to /raid/workspace/jbpark/FaceRecognition/checkpoints/w_tfidentity/ms1m_efficientnet_check/SGD/e_1_l_37.47140884399414.ckpt
 3000/36391 [=>............................] - ETA: 1:31:18 - loss: 37.4735
Epoch 00001: saving model to /raid/workspace/jbpark/FaceRecognition/checkpoints/w_tfidentity/ms1m_efficientnet_check/SGD/e_1_l_37.45500183105469.ckpt
 4000/36391 [==>...........................] - ETA: 1:28:00 - loss: 37.4906
Epoch 00001: saving model to /raid/workspace/jbpark/FaceRecognition/checkpoints/w_tfidentity/ms1m_efficientnet_check/SGD/e_1_l_37.61110305786133.ckpt
 5000/36391 [===>..........................] - ETA: 1:24:30 - loss: 37.5222
Epoc

36391/36391 [==============================] - 6193s 170ms/step - loss: 37.0863 - val_loss: 33.0797
Epoch 2/20
  609/36391 [..............................] - ETA: 1:34:45 - loss: 32.9403
Epoch 00002: saving model to /raid/workspace/jbpark/FaceRecognition/checkpoints/w_tfidentity/ms1m_efficientnet_check/SGD/e_2_l_32.88595199584961.ckpt
 1609/36391 [>.............................] - ETA: 1:32:08 - loss: 32.8780
Epoch 00002: saving model to /raid/workspace/jbpark/FaceRecognition/checkpoints/w_tfidentity/ms1m_efficientnet_check/SGD/e_2_l_32.78194808959961.ckpt
 2609/36391 [=>............................] - ETA: 1:29:38 - loss: 32.8163
Epoch 00002: saving model to /raid/workspace/jbpark/FaceRecognition/checkpoints/w_tfidentity/ms1m_efficientnet_check/SGD/e_2_l_32.65180969238281.ckpt
 3609/36391 [=>............................] - ETA: 1:26:57 - loss: 32.7533
Epoch 00002: saving model to /raid/workspace/jbpark/FaceRecognition/checkpoints/w_tfidentity/ms1m_efficientnet_check/SGD/e_2_l_32.53028